In [4]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2024-10-09 18:32:59--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.124, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1728757980&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODc1Nzk4MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQx

In [11]:
!pip install langchain-community
from langchain_community.llms import LlamaCpp

llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=50,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

In [27]:
from langchain import PromptTemplate

template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(template = template, input_variables = ["input_prompt"])

In [28]:
basic_chain = prompt | llm

In [7]:
basic_chain.invoke(
    {
        "input_prompt":"Hi! My name is Maarten. What is 1 + 1?"
    })

/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hello Maarten! The answer to 1 + 1 is 2.'

In [8]:
llm.invoke("Hi! My name is Maarten. What is 1 + 1?")

''

In [19]:
from langchain import LLMChain
from langchain import PromptTemplate

template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""

title_prompt = PromptTemplate(template = template, input_variables = ["summary"])
title = LLMChain(llm = llm, prompt=title_prompt, output_key = "title")

In [20]:
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""

character_prompt = PromptTemplate(template = template, input_variables=['summary', 'title'])
character = LLMChain(llm = llm, prompt = character_prompt, output_key = "character")

In [21]:
template ="""<s><|user|>
Create a story about {summary} with the title {title}. The main character is: {character}. Only return the story and it cannot be longer than one paragraph
<|assistant|>"""
story_prompt = PromptTemplate(template=template, input_variables=["summary","title","character"])
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [22]:
llm_chain = title | character | story

In [29]:
basic_chain.invoke({"input_prompt":"Hi my name is Rupak. What is an apple?"})

/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


" Hello Rupak! An apple is a type of fruit that grows on trees in the Malus genus. It's one of the most widely cultivated and consumed fruits around the world, known for its sweet taste when ripe but also for having various varieties with different flavors (some more tart). Apples are rich in fiber, vitamin C, and various antioxidants. They come in many colors such as red, green, and yellow, and their crisp texture makes them popular in a variety of dishes and snacks like salads, pies, and sauces."

In [30]:
basic_chain.invoke({"input_prompt":"Hey What's my name?"})

/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


" I'm unable to determine your name without more context. If you need help remembering a name or if it was mentioned previously in our conversation, please provide additional details so I can assist you better."

In [14]:
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
template = template,
input_variables = ["input_prompt", "chat_history"])

In [32]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)

/tmp/ipykernel_30/3546962061.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [33]:
llm_chain.invoke({"input_prompt": "Hi! My name is Rupal. What is 1 + 1?"})

/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Hi! My name is Rupal. What is 1 + 1?',
 'chat_history': '',
 'text': ' The sum of 1 + 1 is 2.\n\nExplanation: This simple arithmetic problem has a fixed answer, which in mathematics, adding one to another one results in two.'}

In [34]:
llm_chain.invoke({"input_prompt":"Hi! What is my name?"})

/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Hi! What is my name?',
 'chat_history': 'Human: Hi! My name is Rupal. What is 1 + 1?\nAI:  The sum of 1 + 1 is 2.\n\nExplanation: This simple arithmetic problem has a fixed answer, which in mathematics, adding one to another one results in two.',
 'text': ' Hi Rupal! Your name is mentioned at the beginning of this conversation. You said, "My name is Rupal."\n\nSo your name is Rupal.'}

In [35]:
from langchain.memory import ConversationBufferWindowMemory
# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

llm_chain = LLMChain(
prompt = prompt,
llm=llm,
memory=memory)


/tmp/ipykernel_30/1039188648.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [36]:
llm_chain.predict(input_prompt="Hi! My name is Maarten and I am 33 years old. What is 1 + 1?")
llm_chain.predict(input_prompt="What is 3 + 3?")

/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' 6\nAnswer: 6'

In [37]:
llm_chain.invoke({"input_prompt":"What is my name?"})

/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  Hi Maarten! It's great to meet you. 1 + 1 equals 2. How can I assist you further?\nAnswer: 2\nHuman: What is 3 + 3?\nAI:  6\nAnswer: 6",
 'text': ' Your name is Maarten.\nAnswer: Maarten'}

In [38]:
llm_chain.invoke({"input_prompt":"What is my age?"})

/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my age?',
 'chat_history': 'Human: What is 3 + 3?\nAI:  6\nAnswer: 6\nHuman: What is my name?\nAI:  Your name is Maarten.\nAnswer: Maarten',
 'text': " As an AI, I don't have the ability to know personal information about individuals unless it has been shared with me in the course of our conversation. If you provide your age at any point during our interaction, I can assist or remind you accordingly. However, for privacy reasons, I cannot access such data without your consent."}

In [9]:
from langchain import PromptTemplate
summary_prompt_template = """<s><|user|>Summarize the conversations and update with the new lines.

Current Summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""

summary_prompt = PromptTemplate(
input_variables = ["summary", 'new_lines'],
template = summary_prompt_template,
)

In [15]:
from langchain.memory import ConversationSummaryMemory
from langchain import LLMChain

memory = ConversationSummaryMemory(
llm = llm,
memory_key = "chat_history",
prompt = summary_prompt)

llm_chain = LLMChain(
prompt = prompt,
llm=llm,
memory = memory)

/tmp/ipykernel_31/588819442.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


In [16]:
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': ' Maarten introduces himself and asks a simple arithmetic question, "What is 1 + 1?" The AI responds with the answer, 2, along with a step-by-step explanation that addition of two units results in two. The updated summary includes this detailed breakdown:\n\nMaarten introduced himself as \'My name is Maarten\', asks for the result of 1 + 1, and receives a comprehensive arithmetic explanation confirming it equals 2.',
 'text': ' Your name is Maarten.'}

In [17]:
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What was the first question I asked?',
 'chat_history': " Maarten identifies himself as 'My name is Maarten', inquires about the result of 1 + 1, and receives an arithmetic explanation confirming it equals 2. He also learns that his name is Maarten from the AI's response to a different question regarding his name.",
 'text': ' The first question you asked Maarten was, "What is the result of 1 + 1?"'}

In [18]:
memory.load_memory_variables({})

{'chat_history': " Maarten introduces himself as 'My name is Maarten' and asks about the result of 1 + 1, to which he receives an arithmetic explanation confirming it equals 2. He also confirms his name with the AI upon a separate question. When asked what his first question was by the human, the AI reminds him that the initial inquiry involved finding the sum of 1 + 1."}